In [1]:
!pip3 install --upgrade langchain langchain_experimental openai --quiet 
!pip3 install --upgrade matplotlib --quiet
!pip3 install --upgrade pandas --quiet
!pip3 install --upgrade sqlalchemy --quiet


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip


In [121]:
import os
import csv
import pandas as pd
from datetime import datetime
from langchain_openai import OpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
import sqlalchemy

In [9]:
# initialise sql engine
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/full_iseller_data.db')

In [10]:
# import csv and convert to db
iseller_data = pd.read_excel('/Users/davidrinaldi/Documents/Projects/myk-dashboard/data/full_iseller_data.xlsx')
iseller_data.to_sql('orders', conn, if_exists='replace', index=False)

62933

In [11]:
db = SQLDatabase.from_uri("sqlite:///data/full_iseller_data.db")


In [85]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI, OpenAI

query_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9)
analysis_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

## location table

In [75]:
# initialise location tables
from scripts.utils import generate_location_df

conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp.db')
location_df = generate_location_df(iseller_data)
location_df.to_sql('locations', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp.db")


In [56]:
location_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [66]:
response = location_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


locations
Invoking: `sql_db_schema` with `{'table_names': 'locations'}`



CREATE TABLE locations (
	outlet_name TEXT, 
	year BIGINT, 
	month_name TEXT, 
	month BIGINT, 
	quantity BIGINT, 
	quantity_diff FLOAT, 
	growth_rate FLOAT
)

/*
3 rows from locations table:
outlet_name	year	month_name	month	quantity	quantity_diff	growth_rate
Ayam! Alun-Alun BKS	2024	March	3	172	None	None
Ayam! Alun-Alun BKS	2024	April	4	134	-38.0	-22.093023255813954
Ayam! Alun-Alun BKS	2024	May	5	23	-111.0	-82.83582089552239
*/1. What is the total quantity sold for each outlet in a specific year?
2. How does the growth rate vary across different months for a particular outlet?
3. Can you provide a trend analysis of quantity changes over the years for all outlets?
<EOF>

> Finished chain.
1. What is the total quantity sold for each outlet in a specific year?
2. How does the growth rate vary across different months for a part

In [110]:
location_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
location_questions

['1. What is the total quantity sold for each outlet in a specific year?',
 '2. How does the growth rate vary across different months for a particular outlet?',
 '3. Can you provide a trend analysis of quantity changes over the years for all outlets?']

In [114]:
# create location answer model
location_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [115]:
location_responses = []
for question in location_questions:
    print(question)
    response = location_query.invoke(question)
    location_responses.append(response['output'])


1. What is the total quantity sold for each outlet in a specific year?
2. How does the growth rate vary across different months for a particular outlet?
3. Can you provide a trend analysis of quantity changes over the years for all outlets?


In [118]:
print(location_responses[0])
    

The total quantity sold for each outlet in the year 2024 is as follows:
1. Ayam! Alun-Alun BKS: 329
2. Ayam! Dapur Kitchen BKS: 1929
3. Ayam! Libersa Park BKS: 347
4. Ayam! RSIJ: 412
5. Ayam! Tambun BKS: 193
6. Bandung: 2264
7. Bekasi - Perjuangan: 39876
8. Cibubur: 14186
9. Jakbar - Taman Palem: 8531
10. Jaksel - Pondok Indah: 14574


In [119]:
print(location_responses[1])


The growth rate for the outlet "Ayam! Alun-Alun BKS" varies across different months as follows:
- In March, the growth rate is not available.
- In April, the growth rate is -22.09%.
- In May, the growth rate is -82.84%.


In [120]:
print(location_responses[2])


The total quantity of products sold across all outlets in the year 2024 was 100,228.


In [138]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/location.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(location_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(location_questions[idx]), 
                escape_newlines(location_responses[idx])
            ]
        )

## sales table

In [142]:
# initialise sales tables
from scripts.utils import generate_sales_table

conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp2.db')
sales_df = generate_sales_table(iseller_data)
sales_df.to_sql('sales', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp2.db")

In [143]:
sales_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [158]:
response = sales_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


sales
Invoking: `sql_db_schema` with `{'table_names': 'sales'}`



CREATE TABLE sales (
	clean_product_name TEXT, 
	product_type TEXT, 
	total_revenue BIGINT, 
	current_quarter_revenue FLOAT, 
	last_quarter_revenue FLOAT, 
	current_month_revenue FLOAT, 
	last_month_revenue FLOAT, 
	quarterly_growth FLOAT, 
	monthly_growth FLOAT, 
	"quarter_Growth(%)" FLOAT, 
	"Monthly_Growth(%)" FLOAT
)

/*
3 rows from sales table:
clean_product_name	product_type	total_revenue	current_quarter_revenue	last_quarter_revenue	current_month_revenue	last_month_revenue	quarterly_growth	monthly_growth	quarter_Growth(%)	Monthly_Growth(%)
(es batu) crystalline cup	REGULAR	201033098	55569433.0	145463665.0	10521000.0	45048433.0	-61.79840993281724	-76.64513658000047	-0.45	-0.62
a&w root beer kaleng	REGULAR	21235100	5086300.0	16148800.0	89000.0	4997300.0	-68.50354205885267	-98.21903828067157	-0.52	-0.97
absolut vodka	VODKA	515999

In [159]:
sales_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
sales_questions

['1. What is the distribution of total revenue across different product types?',
 '2. How does the quarterly growth compare between regular products and vodka products?',
 '3. Can you provide insights on the revenue trend over the current quarter for the top-selling products?']

In [160]:
# create location answer model
sales_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [161]:
sales_responses = []
for question in sales_questions:
    print(question)
    response = sales_query.invoke(question)
    sales_responses.append(response['output'])

1. What is the distribution of total revenue across different product types?
2. How does the quarterly growth compare between regular products and vodka products?
3. Can you provide insights on the revenue trend over the current quarter for the top-selling products?


In [162]:
print(sales_responses[0])

The distribution of total revenue across different product types is as follows:

- ANGGUR: 2,444,051,700
- BEER: 2,623,261,150
- COGNAC: 230,876,700
- EMBALASE: 4,920,000
- FOOD: 103,646,882
- FREE: 400,021
- GIN: 401,647,900
- LIQUOR: 1,326,538,400
- MAKGEOLLI: 103,197,400
- MERCHANDISE: 51,725,300
- MEZCAL: 1,075,000
- REGULAR: 880,910,220
- ROKOK: 24,531,000
- RUM: 499,287,400
- SOJU: 1,787,271,100
- SPIRIT: 68,981,900
- TEQUILA: 122,984,100
- VAPE: 1,775,000
- VODKA: 1,347,021,116
- WHISKY: 2,721,392,800
- WINE: 997,531,400


In [163]:
print(sales_responses[1])

The average quarterly growth for regular products is -64.51% and for vodka products is -56.33%.


In [164]:
print(sales_responses[2])

The top-selling products for the current quarter and their respective revenues are as follows:

1. Jameson (WHISKY) - Current Quarter Revenue: $181,886,500
2. Kawa Kawa Anggur Hijau (ANGGUR) - Current Quarter Revenue: $169,912,400
3. Api Anggur Hijau (ANGGUR) - Current Quarter Revenue: $143,084,400
4. Iceland Vodka (VODKA) - Current Quarter Revenue: $120,239,700

These products have shown varying revenue trends over the current quarter. If you would like more detailed insights or specific analysis on any of these products, please let me know.


In [165]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/sales.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(location_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(sales_questions[idx]), 
                escape_newlines(sales_responses[idx])
            ]
        )

## Top Product by Quantity Sold

In [177]:
# Top Product by Quantity Sold
tpqs = iseller_data.groupby(['clean_product_name', 'year', 'month', 'week']).agg({'total_amount': 'mean', 'quantity': 'sum'}).reset_index()\
        .sort_values(by='quantity', ascending=False).sort_values(by=["year", "month", "week"])
tpqs.head()

,clean_product_name,year,month,week,total_amount,quantity
3595,kawa kawa anggur hijau,2024,1,1,140173.195980,557
340,anggur merah gold,2024,1,1,131484.556314,395
457,api anggur hijau,2024,1,1,164322.362245,292
296,anggur merah,2024,1,1,118656.692308,231
1125,bintang kaleng,2024,1,1,222735.696429,224


In [191]:
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp3.db')
tpqs.to_sql('top_product_by_quantity_sold', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp3.db")

In [193]:
tpqs_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [199]:
response = tpqs_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


top_product_by_quantity_sold
Invoking: `sql_db_schema` with `{'table_names': 'top_product_by_quantity_sold'}`



CREATE TABLE top_product_by_quantity_sold (
	clean_product_name TEXT, 
	year BIGINT, 
	month BIGINT, 
	week BIGINT, 
	total_amount FLOAT, 
	quantity BIGINT
)

/*
3 rows from top_product_by_quantity_sold table:
clean_product_name	year	month	week	total_amount	quantity
kawa kawa anggur hijau	2024	1	1	140173.1959798995	557
anggur merah gold	2024	1	1	131484.55631399318	395
api anggur hijau	2024	1	1	164322.36224489796	292
*/1. What are the top products by total amount sold in a specific year?
2. Can you provide the distribution of quantities sold for different products across months and weeks?
3. How does the total amount sold vary for products with different names? 
<EOF>

> Finished chain.
1. What are the top products by total amount sold in a specific year?
2. Can you provide the distributi

In [200]:
tpqs_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
tpqs_questions

['1. What are the top products by total amount sold in a specific year?',
 '2. Can you provide the distribution of quantities sold for different products across months and weeks?',
 '3. How does the total amount sold vary for products with different names? ']

In [201]:
# create location answer model
tpqs_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [202]:
tpqs_responses = []
for question in tpqs_questions:
    print(question)
    response = tpqs_query.invoke(question)
    tpqs_responses.append(response['output'])

1. What are the top products by total amount sold in a specific year?
2. Can you provide the distribution of quantities sold for different products across months and weeks?
3. How does the total amount sold vary for products with different names? 


In [203]:
print(tpqs_responses[0])

The top products by total amount sold in the year 2024 are as follows:
1. Product: Hibiki Japanese Harmony, Total Amount Sold: $5,377,000.00
2. Product: Hibiki Japanese Harmony, Total Amount Sold: $4,887,000.00
3. Product: Nikka Whisky From The Barrel Japanese Whisky, Total Amount Sold: $4,545,016.00
4. Product: Ballantines Blended Scotch Whisky, Total Amount Sold: $4,545,016.00
5. Product: Jack Daniel's No.7, Total Amount Sold: $4,259,000.00
6. Product: Hennessy VSOP, Total Amount Sold: $4,196,000.00
7. Product: Monkey Shoulder, Total Amount Sold: $3,947,000.00
8. Product: Sababay Mascetti, Total Amount Sold: $3,944,000.00
9. Product: Roku Gin, Total Amount Sold: $3,937,000.00
10. Product: Martell VSOP, Total Amount Sold: $3,832,488.00


In [204]:
print(tpqs_responses[1])

The distribution of quantities sold for different products across months and weeks is as follows:

1. Product: "kawa kawa anggur hijau", Year: 2024, Month: 1, Week: 1, Quantity Sold: 557
2. Product: "anggur merah gold", Year: 2024, Month: 1, Week: 1, Quantity Sold: 395
3. Product: "api anggur hijau", Year: 2024, Month: 1, Week: 1, Quantity Sold: 292
4. Product: "anggur merah", Year: 2024, Month: 1, Week: 1, Quantity Sold: 231
5. Product: "bintang kaleng", Year: 2024, Month: 1, Week: 1, Quantity Sold: 224
6. Product: "(es batu) crystalline cup", Year: 2024, Month: 1, Week: 1, Quantity Sold: 205
7. Product: "draft beer kaleng", Year: 2024, Month: 1, Week: 1, Quantity Sold: 202
8. Product: "anggur ginseng intisari", Year: 2024, Month: 1, Week: 1, Quantity Sold: 192
9. Product: "bintang botol", Year: 2024, Month: 1, Week: 1, Quantity Sold: 181
10. Product: "iceland vodka", Year: 2024, Month: 1, Week: 1, Quantity Sold: 173

These are the top 10 products with their quantities sold across dif

In [205]:
print(tpqs_responses[2])

The total amount sold varies for products with different names. Here are the top 10 products based on the total amount sold:

1. Hennessy VSOP - $35,864,390.67
2. Martell VSOP - $30,219,499.59
3. Hibiki Japanese Harmony - $24,804,258.00
4. Glenfiddich 12yo - $24,730,651.99
5. JW Black Label - $22,612,401.42
6. Chivas 12yo - $22,505,771.37
7. Grey Goose - $21,013,684.14
8. Singleton Glen Ord 12yo - $20,713,785.84
9. Singleton Luscious Nectar 12yo - $20,285,419.99
10. Glenlivet 12yo - $18,603,922.42
